# Build Deep Research

This notebook walks through a naive implementation of DeepResearch like behaviour using ADK.


<table align="left">
  <td style="text-align: center">
    <a href="https://colab.sandbox.google.com/github/hupili/google-adk-101/blob/main/Build_Deep_Research.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2Fhupili%2Fgoogle-adk-101%2Fmain%2FBuild_Deep_Research.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-enterprise-logo.png" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://github.com/hupili/google-adk-101/blob/main/Build_Deep_Research.ipynb">
      <img width="32px" src="https://www.gstatic.com/monospace/240802/git_host_github_mask.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/hupili/google-adk-101/blob/main/Build_Deep_Research.ipynb">
      <img width="32px" src="https://www.gstatic.com/cloud/images/navigation/vertex-ai.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>

| | |
|-|-|
| Author(s) | [Pili Hu](https://github.com/hupili)|
| Reviewers(s) | Nil |
| Last updated | 2025-06-02 |
| google-adk | 1.1.1 |
| vertexai | 1.93.1 |

## Environment

In [1]:
import os

# API_KEY = '' # @param {type:"string"}
# os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE" # Use Vertex AI API
# os.environ["GOOGLE_API_KEY"] = API_KEY

PROJECT_ID = "hupili-genai-bb"  # @param {type:"string"}
if not PROJECT_ID:
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = "us-central1" # @param {type:"string"}

STAGING_BUCKET = 'gs://agent_engine_deploy_staging' # @param {type:"string"}

os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "TRUE" # Use Vertex AI API

# [your-project-id]

In [2]:
import vertexai

vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)

In [3]:
from google.colab import auth
auth.authenticate_user()

In [4]:
!pip install google-adk==1.1.1

In [5]:
from google import adk

In [6]:
adk.__version__

'1.1.1'

In [7]:
vertexai.__version__

'1.95.0'

## Util functions

In [8]:
from vertexai import agent_engines
from vertexai.preview import reasoning_engines

In [9]:
def delete(resource_id: str) -> None:
    remote_agent = agent_engines.get(resource_id)
    remote_agent.delete(force=True)
    print(f"Deleted remote agent: {resource_id}")

def list_agents() -> None:
    remote_agents = agent_engines.list()
    TEMPLATE = '''
{agent.name} ("{agent.display_name}")
- Create time: {agent.create_time}
- Update time: {agent.update_time}
'''
    remote_agents_string = '\n'.join(TEMPLATE.format(agent=agent) for agent in remote_agents)
    print(f"All remote agents:\n{remote_agents_string}")

In [10]:
list_agents()

All remote agents:

5147799092037943296 ("")
- Create time: 2025-06-02 14:14:28.600149+00:00
- Update time: 2025-06-02 14:18:55.955685+00:00


2164164338904989696 ("")
- Create time: 2025-06-02 11:57:47.050939+00:00
- Update time: 2025-06-02 12:01:18.059991+00:00


6087608854902734848 ("DICE_AGENT")
- Create time: 2025-04-25 08:19:20.817490+00:00
- Update time: 2025-04-25 08:22:05.936293+00:00


7916070303615156224 ("llm_auditor")
- Create time: 2025-04-25 07:13:25.339138+00:00
- Update time: 2025-04-25 07:18:12.898631+00:00


5307078744484085760 ("llm_auditor")
- Create time: 2025-04-09 08:22:36.179879+00:00
- Update time: 2025-04-09 08:25:42.734584+00:00


8402318325882814464 ("")
- Create time: 2025-03-28 08:42:23.121480+00:00
- Update time: 2025-03-28 08:46:29.629436+00:00


66410502317670400 ("my-awesome-agent")
- Create time: 2025-03-07 01:22:05.930759+00:00
- Update time: 2025-03-07 01:33:02.201959+00:00



In [11]:
def call_remote_agent_engine_app(app, user_id, session_id, query):
  response_text = ''
  for event in app.stream_query(
      user_id=user_id,
      session_id=session_id,
      message=query,
  ):
      print(event)
      all_parts = event.get('content', {}).get('parts', [])
      for part in all_parts:
          response_text += (part.get('text', ''))
  return response_text

## Simple Search Agent

In [12]:
from google.adk.tools import google_search
from google.adk import Agent
from google.adk.agents import Agent, LlmAgent
from google.genai import types
from pydantic import BaseModel
from google.genai import types
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner

MODEL = "gemini-2.0-flash"
APP_NAME = "simple_search_example"
USER_ID = "user12345"
SESSION_ID = "session12345"
AGENT_NAME = "simple_search_agent"

# Agent
simple_search_agent = Agent(
    model=MODEL,
    name="simple_search_agent",
    description="Agent to answer questions using Google Search.",
    instruction="I can answer your questions by searching the internet. Just ask me anything!",
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=8000,
    ),
    tools=[google_search],
)

app = reasoning_engines.AdkApp(
    agent=simple_search_agent,
    enable_tracing=True,
)

In [14]:
session = app.create_session(user_id=USER_ID)
session

Session(id='a45543d2-7d66-42fe-9105-028cf6e8a071', app_name='default-app-name', user_id='user12345', state={}, events=[], last_update_time=1748878780.263915)

In [15]:
call_remote_agent_engine_app(app, USER_ID, session.id, "what is the weather in New York")

{'content': {'parts': [{'text': 'The weather in New York City, NY on Monday, June 2, 2025 is sunny with a temperature of 67°F (19°C), but it feels like 71°F (22°C). There is a 0% chance of rain.\n\nThe forecast for the next few days is:\n\n*   **Tuesday:** Sunny, with a high near 80°F (26°C).\n*   **Wednesday:** Sunny, with a high near 85°F (28°C).\n*   **Thursday:** Mostly sunny, with a high near 87°F (33°C) and a 30% chance of showers after 2pm.\n*   **Friday:** Partly sunny, with a high near 86°F (28°C) and a 30% chance of showers.\n'}], 'role': 'model'}, 'grounding_metadata': {'grounding_chunks': [{'web': {'domain': 'google.com', 'title': 'Weather information for locality: New York, administrative_area: NY', 'uri': 'https://www.google.com/search?q=weather+in+New York,+NY'}}, {'web': {'domain': 'bbc.com', 'title': 'bbc.com', 'uri': 'https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXFhaqQGQGybtEZ2uOnNs2onQDsOwI2sRMVOf0EO5BM7AzQXU06Fk1zL5P7ztxVtlzaZa7vFAJKJ2mzpZtH-C

'The weather in New York City, NY on Monday, June 2, 2025 is sunny with a temperature of 67°F (19°C), but it feels like 71°F (22°C). There is a 0% chance of rain.\n\nThe forecast for the next few days is:\n\n*   **Tuesday:** Sunny, with a high near 80°F (26°C).\n*   **Wednesday:** Sunny, with a high near 85°F (28°C).\n*   **Thursday:** Mostly sunny, with a high near 87°F (33°C) and a 30% chance of showers after 2pm.\n*   **Friday:** Partly sunny, with a high near 86°F (28°C) and a 30% chance of showers.\n'

In [16]:
call_remote_agent_engine_app(app, USER_ID, session.id, "is earth square?")

{'content': {'parts': [{'text': "No, the Earth is not square. While it appears round from space, it's more accurately described as an irregularly shaped ellipsoid, or a sphere.\n\nHere's why:\n\n*   **Gravity:** Earth's gravity pulls everything towards its center. If Earth were made of a homogenous material, gravity would make it a perfect sphere.\n*   **Rotation:** The Earth's rotation causes a centrifugal force, which is greater at the equator than at the poles. This force deforms the sphere into an oblate ellipsoid, which means it bulges at the equator.\n*   **Irregularities:** The Earth isn't homogenous. It comprises materials with different densities, and it has mountains and ocean trenches, all of which contribute to its irregular shape.\n"}], 'role': 'model'}, 'grounding_metadata': {'grounding_chunks': [{'web': {'domain': 'ucl.ac.uk', 'title': 'ucl.ac.uk', 'uri': 'https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXE5NgBud-1EB-T8NkQYrvIDyJXwgGJy63LGMSAfqe4r5EXiu

"No, the Earth is not square. While it appears round from space, it's more accurately described as an irregularly shaped ellipsoid, or a sphere.\n\nHere's why:\n\n*   **Gravity:** Earth's gravity pulls everything towards its center. If Earth were made of a homogenous material, gravity would make it a perfect sphere.\n*   **Rotation:** The Earth's rotation causes a centrifugal force, which is greater at the equator than at the poles. This force deforms the sphere into an oblate ellipsoid, which means it bulges at the equator.\n*   **Irregularities:** The Earth isn't homogenous. It comprises materials with different densities, and it has mountains and ocean trenches, all of which contribute to its irregular shape.\n"

## Search multiple questions


In [18]:
from google.adk.tools.agent_tool import AgentTool

# Instantiate again because each agent tree shall have unique objects.
simple_search_agent = Agent(
    model=MODEL,
    name="simple_search_agent",
    description="Agent to answer questions using Google Search.",
    instruction="I can answer your questions by searching the internet. Just ask me anything!",
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=8000,
    ),
    tools=[google_search],
)

multiple_search_agent = Agent(
    model=MODEL,
    name="research_agent",
    description="Get answers to multiple questions one by one.",
    instruction="Observe the pending questions in the chat log and call the simple_search_agent to conduct search and give answers to each question. Present the search result in a markdown, one section per question and answer",
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=8000,
    ),
    tools=[AgentTool(simple_search_agent)],
)

app = reasoning_engines.AdkApp(
    agent=multiple_search_agent,
    enable_tracing=True,
)

session = app.create_session(user_id=USER_ID)

call_remote_agent_engine_app(app, USER_ID, session.id, """
- Is Earth square?
- Is Moon flat?
- Is Sun cold?
""")

{'content': {'parts': [{'function_call': {'id': 'adk-9a69d478-19fd-4851-9c21-58233dcde773', 'args': {'request': 'Is Earth square?'}, 'name': 'simple_search_agent'}}, {'function_call': {'id': 'adk-7dbfd77b-5c0b-49e6-8c14-23b94531cbec', 'args': {'request': 'Is Moon flat?'}, 'name': 'simple_search_agent'}}, {'function_call': {'id': 'adk-bf91f868-96ae-469d-9299-a6e4670afcc0', 'args': {'request': 'Is Sun cold?'}, 'name': 'simple_search_agent'}}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 30, 'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 30}], 'prompt_token_count': 107, 'prompt_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 107}], 'total_token_count': 137, 'traffic_type': <TrafficType.ON_DEMAND: 'ON_DEMAND'>}, 'invocation_id': 'e-35b48e17-4858-4326-8ace-513eb1fc3c85', 'author': 'research_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids'

{'content': {'parts': [{'function_response': {'id': 'adk-9a69d478-19fd-4851-9c21-58233dcde773', 'name': 'simple_search_agent', 'response': {'result': "No, the Earth is not square. While it appears round from space, it is more accurately described as an irregularly shaped ellipsoid, sometimes referred to as a spheroid.\n\nHere's why:\n\n*   **Gravity:** The Earth's own gravity pulls everything towards its center. If the Earth were made of a homogenous material, it would be a perfect sphere, flattened slightly by its rotation.\n*   **Rotation:** The Earth's rotation causes a centrifugal force that makes it bulge at the equator.\n*   **Irregularities:** The Earth is not perfectly uniform in density or composition. Mountains, ocean trenches, and variations in the Earth's gravity field further distort its shape.\n\n"}}}, {'function_response': {'id': 'adk-7dbfd77b-5c0b-49e6-8c14-23b94531cbec', 'name': 'simple_search_agent', 'response': {'result': "No, the Moon is not flat. It is a sphere, al

{'content': {'parts': [{'text': '### Is Earth square?\n\nNo, the Earth is not square. It is an irregularly shaped ellipsoid, sometimes referred to as a spheroid.\n\n### Is Moon flat?\n\nNo, the Moon is not flat. It is a sphere, although not a perfect one, and is often described as an oblate spheroid or even egg-shaped.\n\n### Is Sun cold?\n\nThe Sun is not cold. It is a giant ball of fire propelled by nuclear reactions, with a surface temperature of about 5800 Kelvin (around 10,000 degrees Fahrenheit).\n'}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 121, 'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 121}], 'prompt_token_count': 505, 'prompt_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 505}], 'total_token_count': 626, 'traffic_type': <TrafficType.ON_DEMAND: 'ON_DEMAND'>}, 'invocation_id': 'e-35b48e17-4858-4326-8ace-513eb1fc3c85', 'author': 'research_agent', 'actions': {'state_delta': {}, 'artif

'### Is Earth square?\n\nNo, the Earth is not square. It is an irregularly shaped ellipsoid, sometimes referred to as a spheroid.\n\n### Is Moon flat?\n\nNo, the Moon is not flat. It is a sphere, although not a perfect one, and is often described as an oblate spheroid or even egg-shaped.\n\n### Is Sun cold?\n\nThe Sun is not cold. It is a giant ball of fire propelled by nuclear reactions, with a surface temperature of about 5800 Kelvin (around 10,000 degrees Fahrenheit).\n'

In [19]:
# One sample data related to function call
# data = {
#     'content': {
#         'parts': [
#             {'function_call':
#              {'id': 'adk-86084634-fdb5-45ff-be8f-ef3fc5866a35',
#               'args': {'request': 'Is Earth square?'},
#               'name': 'simple_search_agent'}},
#             {'function_call':
#              {'id': 'adk-482d0901-8ff9-4f48-bf2c-03539b6dc35a',
#               'args': {'request': 'Is Moon flat?'}, 'name': 'simple_search_agent'}},
#             {'function_call':
#              {'id': 'adk-a8b821f0-b93c-470e-bf41-fa89e7be6d9c',
#               'args': {'request': 'Is Sun cold?'}, 'name': 'simple_search_agent'}}

## Generate the research plan

In [20]:
DEEP_RESEARCH_PLAN_INSTRUCTION = """
Instruction for a Deep Research Plan Agent
As a Deep Research Plan Agent, your primary objective is to generate a comprehensive and insightful research plan. This plan will be presented as a markdown bullet point list of questions. The goal is to not only address the user's explicit query but also to anticipate and explore related avenues of inquiry that would be of interest for a thorough understanding of the topic. Your output should guide the user through a structured and deep exploration of their subject.

Here is the process to follow:

Deconstruct the Core Topic:

What is the fundamental subject of the user's request?
What are the primary keywords and concepts?
What is the implicit goal of the user's research? (e.g., to understand a concept, to make a decision, to compare and contrast, to solve a problem).
Generate Foundational Questions:

Start with the basics to establish a strong foundation.
What is the definition of the core topic?
What is its history and origin?
What are the key principles and theories that underpin it?
Who are the seminal thinkers, researchers, or figures associated with this topic?
Explore Deeper, More Nuanced Aspects:

Move beyond the surface-level inquiries.
What are the current debates or controversies surrounding this topic?
What are the different schools of thought or perspectives?
What are the key components, mechanisms, or processes involved? How do they interact?
What are the less obvious or often overlooked aspects of this topic?
Investigate Connections and Relationships:

Think about how the topic connects to broader fields and concepts.
How does this topic relate to other relevant disciplines or areas of study?
What are the upstream causes and downstream effects of the subject?
How has this topic influenced, or been influenced by, historical, social, technological, or economic factors?
Consider Practical Applications and Implications:

Explore the real-world significance of the topic.
What are the practical applications or use cases?
What are the ethical, social, or economic implications?
Who are the key stakeholders affected by or involved in this topic?
Look to the Future:

Encourage forward-thinking inquiry.
What are the emerging trends and future directions of research in this area?
What are the unanswered questions or existing gaps in knowledge?
What are the potential future impacts or developments related to this topic?
Synthesize and Structure the Plan:

Organize the generated questions into a logical flow. A good structure might be:
Foundational Understanding
In-Depth Analysis
Context and Connections
Practical Implications
Future Outlook
Present the final output as a clean, easy-to-read markdown bullet point list.
Use sub-bullets to group related questions and create a hierarchical structure.
Ensure the questions are open-ended and encourage critical thinking rather than simple fact-finding.
By following these instructions, you will create a research plan that is not only deep and thoughtful but also anticipates the user's latent interests, guiding them toward a truly comprehensive understanding of their chosen topic.
"""

In [21]:
# Instantiate again because each agent tree shall have unique objects.
plan_agent = Agent(
    model=MODEL,
    name="plan_agent",
    description="Agent to generate a research plan based on the question or topic given by the user",
    instruction=DEEP_RESEARCH_PLAN_INSTRUCTION,
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=8000,
    ),
)

In [23]:
app = reasoning_engines.AdkApp(
    agent=plan_agent,
    enable_tracing=True,
)

session = app.create_session(user_id=USER_ID)

response_text = call_remote_agent_engine_app(app, USER_ID, session.id, """AI Agent's impact on labour market""")

{'content': {'parts': [{'text': 'Okay, I can create a research plan to deeply investigate the impact of AI Agents on the labor market. Here\'s a structured plan to guide a comprehensive exploration of this topic:\n\n*   **Foundational Understanding:**\n\n    *   What exactly are "AI Agents," and how do they differ from other forms of AI or automation?\n        *   What are the different types of AI Agents (e.g., task-specific, general-purpose, embodied, software-based)?\n        *   What are some examples of AI Agents currently being used in different industries?\n    *   What are the key characteristics of the current labor market that are relevant to the impact of AI Agents?\n        *   What are the existing trends in automation, globalization, and skills gaps?\n        *   How is the labor market structured in terms of different sectors, skill levels, and employment types?\n    *   What are the projected rates of adoption and development of AI Agents in the coming years?\n        *

In [24]:
# prompt: display response_text as markdown

from IPython.display import display, Markdown

display(Markdown(response_text))

Okay, I can create a research plan to deeply investigate the impact of AI Agents on the labor market. Here's a structured plan to guide a comprehensive exploration of this topic:

*   **Foundational Understanding:**

    *   What exactly are "AI Agents," and how do they differ from other forms of AI or automation?
        *   What are the different types of AI Agents (e.g., task-specific, general-purpose, embodied, software-based)?
        *   What are some examples of AI Agents currently being used in different industries?
    *   What are the key characteristics of the current labor market that are relevant to the impact of AI Agents?
        *   What are the existing trends in automation, globalization, and skills gaps?
        *   How is the labor market structured in terms of different sectors, skill levels, and employment types?
    *   What are the projected rates of adoption and development of AI Agents in the coming years?
        *   Which industries are likely to be most heavily impacted in the short and long term?
        *   What are the major technological bottlenecks or enabling factors for wider adoption?

*   **In-Depth Analysis:**

    *   How are AI Agents currently impacting employment levels in different sectors?
        *   Which jobs are most vulnerable to displacement by AI Agents?
        *   Are AI Agents creating new jobs, and if so, what types of jobs are these?
        *   What is the net effect of AI Agents on overall employment rates?
    *   How are AI Agents changing the nature of work and the skills required in different occupations?
        *   Are AI Agents augmenting human capabilities or replacing them entirely?
        *   What new skills are becoming more important in an AI-driven workplace (e.g., creativity, critical thinking, emotional intelligence, AI oversight)?
        *   How is the demand for different skills evolving as AI Agents become more prevalent?
    *   What are the potential impacts of AI Agents on wages and income inequality?
        *   Are AI Agents contributing to wage stagnation or decline for certain groups of workers?
        *   Are the benefits of AI-driven productivity gains being distributed equitably?
        *   Could AI Agents exacerbate existing inequalities based on race, gender, or education level?

*   **Context and Connections:**

    *   How does the impact of AI Agents on the labor market compare to the impact of previous technological revolutions (e.g., the Industrial Revolution, the rise of computers)?
        *   What lessons can we learn from history about managing technological disruption and its effects on employment?
    *   What are the ethical considerations related to the use of AI Agents in the workplace?
        *   How can we ensure fairness, transparency, and accountability in the design and deployment of AI Agents?
        *   What are the potential risks of bias or discrimination in AI-driven hiring, promotion, or performance evaluation processes?
    *   How are government policies and regulations addressing the challenges and opportunities presented by AI Agents in the labor market?
        *   Are existing labor laws and social safety nets adequate to protect workers in an AI-driven economy?
        *   What types of policies are being considered or implemented to promote retraining, upskilling, and job creation?
        *   How are governments addressing issues such as data privacy, algorithmic bias, and the concentration of economic power in the hands of AI developers?

*   **Practical Implications:**

    *   What strategies can businesses adopt to successfully integrate AI Agents into their workforce while minimizing negative impacts on employees?
        *   How can companies invest in employee training and development to prepare workers for new roles and responsibilities?
        *   What are the best practices for managing the ethical and social implications of AI Agent deployment?
    *   What steps can individuals take to prepare themselves for the changing demands of the labor market in the age of AI Agents?
        *   What skills and knowledge are most likely to be valuable in the future?
        *   How can workers adapt to a more dynamic and uncertain job market?
    *   What are the implications of AI Agents for education and training systems?
        *   How should educational institutions adapt their curricula to prepare students for an AI-driven world?
        *   What are the most effective approaches to lifelong learning and reskilling?

*   **Future Outlook:**

    *   What are the potential long-term scenarios for the labor market as AI Agents become more sophisticated and pervasive?
        *   Will AI Agents eventually lead to widespread job displacement and a "jobless future"?
        *   Could AI Agents create new forms of work and economic opportunity that we cannot yet imagine?
    *   What are the key areas of ongoing research and development related to AI Agents and their impact on the labor market?
        *   What are the most pressing research questions that need to be addressed?
        *   What are the potential breakthroughs or unexpected developments that could significantly alter the trajectory of AI's impact on work?
    *   What are the potential societal and political implications of a labor market increasingly shaped by AI Agents?
        *   How might AI Agents affect social cohesion, political stability, and the distribution of power?
        *   What are the long-term implications for democracy and human autonomy?


## Puting it all together

In [25]:
# Instantiate again because each agent tree shall have unique objects.
plan_agent = Agent(
    model=MODEL,
    name="plan_agent",
    description="Agent to generate a research plan based on the question or topic given by the user",
    instruction=DEEP_RESEARCH_PLAN_INSTRUCTION,
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=8000,
    ),
)

simple_search_agent = Agent(
    model=MODEL,
    name="simple_search_agent",
    description="Agent to answer questions using Google Search.",
    instruction="I can answer your questions by searching the internet. Just ask me anything!",
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=8000,
    ),
    tools=[google_search],
)

multiple_search_agent = Agent(
    model=MODEL,
    name="research_agent",
    description="Get answers to multiple questions one by one.",
    instruction="Observe the pending questions in the chat log and call the simple_search_agent to conduct search and give answers to each question. Present the search result in a markdown, one section per question and answer",
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=8000,
    ),
    tools=[AgentTool(simple_search_agent)],
)

report_agent = Agent(
    model=MODEL,
    name="report_agent",
    description="Agent to generate a markdown research report from the questions and their search results from the Internet",
    instruction="Summarize the research results and present research report in markdown format",
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=8000,
    ),
)

from google.adk.agents.sequential_agent import SequentialAgent

root_agent = SequentialAgent(
    name="deep_researcher",
    description="This is a deep research that makes research plan, use Google search to gather information and generate a report.",
    sub_agents=[plan_agent, multiple_search_agent, report_agent],
)

app = reasoning_engines.AdkApp(
    agent=root_agent,
    enable_tracing=True,
)

In [27]:
session = app.create_session(user_id=USER_ID)

response_text = call_remote_agent_engine_app(app, USER_ID, session.id, """AI Agent's impact on labour market""")

{'content': {'parts': [{'text': "Okay, I can create a research plan to investigate the impact of AI Agents on the labor market. Here's a plan designed to guide a thorough exploration of the topic:\n\n*   **Foundational Understanding**\n\n    *   What is the definition of an AI Agent, and what are the different types of AI Agents relevant to the labor market (e.g., chatbots, robotic process automation, autonomous systems)?\n    *   What are the key capabilities and limitations of current AI Agent technologies?\n    *   What are the historical trends of automation and technological advancements and their effects on employment?\n*   **In-Depth Analysis**\n\n    *   How are AI Agents currently being implemented in various industries and sectors?\n    *   What specific job roles and tasks are most susceptible to being augmented or replaced by AI Agents?\n    *   What are the observed effects of AI Agent implementation on employment levels, both positive (job creation) and negative (job disp

{'content': {'parts': [{'text': 'Okay, I will now conduct searches to answer the questions outlined in the research plan. Here are the search results, organized by question:\n\n**Foundational Understanding**\n\n*   **What is the definition of an AI Agent, and what are the different types of AI Agents relevant to the labor market (e.g., chatbots, robotic process automation, autonomous systems)?**\n'}, {'function_call': {'id': 'adk-42c95696-04c6-4ebb-bb38-f04098214038', 'args': {'request': 'definition of AI Agent labor market'}, 'name': 'simple_search_agent'}}, {'function_call': {'id': 'adk-1ae91f4f-9efa-4daf-bca0-b0506ed9909d', 'args': {'request': 'types of AI Agents in labor market'}, 'name': 'simple_search_agent'}}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 102, 'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 102}], 'prompt_token_count': 746, 'prompt_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count'

{'content': {'parts': [{'function_response': {'id': 'adk-42c95696-04c6-4ebb-bb38-f04098214038', 'name': 'simple_search_agent', 'response': {'result': 'The "AI Agent labor market" refers to the evolving ecosystem where AI agents are increasingly integrated into the workforce, impacting job roles, skill requirements, and overall employment trends. AI agents are sophisticated software programs designed to autonomously perform tasks, make decisions, and interact with their environment with minimal human oversight.\n\nHere\'s a breakdown of key aspects:\n\n**1. Definition of AI Agents:**\n\n*   AI agents are software programs designed to autonomously perform tasks or make decisions based on data.\n*   They perceive their environment, process information, and take action, similar to humans but powered by algorithms and machine learning.\n*   They can be simple, like reactive agents responding to specific inputs, or more complex, like deliberative agents that plan and predict outcomes. Hybrid

{'content': {'parts': [{'text': '*   **What are the key capabilities and limitations of current AI Agent technologies?**\n'}, {'function_call': {'id': 'adk-ed45a77d-db6f-423f-bd22-043be5f51b1d', 'args': {'request': 'capabilities of AI Agent technologies'}, 'name': 'simple_search_agent'}}, {'function_call': {'id': 'adk-13b85a22-8bb0-4a47-83a8-480c41555ad9', 'args': {'request': 'limitations of AI Agent technologies'}, 'name': 'simple_search_agent'}}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 40, 'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 40}], 'prompt_token_count': 2217, 'prompt_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 2217}], 'total_token_count': 2257, 'traffic_type': <TrafficType.ON_DEMAND: 'ON_DEMAND'>}, 'invocation_id': 'e-54988485-7b90-4f04-82eb-3d004a64995e', 'author': 'research_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_runnin

{'content': {'parts': [{'function_response': {'id': 'adk-ed45a77d-db6f-423f-bd22-043be5f51b1d', 'name': 'simple_search_agent', 'response': {'result': 'AI Agent technologies have a broad range of capabilities, including:\n\n*   **Reasoning:** AI agents use logic and available information to draw conclusions, make inferences, and solve problems. They can analyze data, identify patterns, and make informed decisions based on evidence and context.\n*   **Acting:** AI agents can perform tasks based on decisions, plans, or external input. This can include physical actions or digital actions like sending messages or updating data.\n*   **Autonomy:** AI agents can operate and make decisions independently to achieve a goal.\n*   **Learning:** AI agents employ machine learning to adapt and improve their performance over time.\n*   **Task Automation: AI agents can automate complex tasks that would otherwise require human resources, leading to goals being reached inexpensively, rapidly, and at scal

...

{'content': {'parts': [{'function_response': {'id': 'adk-bd7a24a2-1e0f-4a93-bb05-73a81204a790', 'name': 'simple_search_agent', 'response': {'result': "It appears you're asking about the uncertainties and unknowns surrounding the future impact of AI agents on the labor market. Here's a breakdown of key areas of concern and potential impacts:\n\n**1. Job Displacement & Creation**\n*   **Automation:** AI agents can automate routine and repetitive tasks, leading to job displacement in certain sectors. Some estimate AI could impact nearly 50 million US jobs.\n*   **New Roles:** AI is also expected to create new jobs, such as AI trainers, data analysts, AI maintenance professionals, AI ethicists, and machine learning engineers.\n*   **Net Impact:** Projections vary, with some suggesting a net gain in jobs. One projection estimates AI displacing 75 million jobs globally but creating 133 million new ones by 2025.\n*   **Entry-Level Roles:** There's concern AI could reduce entry-level opportuni

{'content': {'parts': [{'text': '*   **What research and development efforts are needed to ensure that AI Agents are used in a way that benefits both businesses and workers?**\n'}, {'function_call': {'id': 'adk-1d5c6a89-edcc-4a2c-8ccb-62533dcd524c', 'args': {'request': 'research and development efforts are needed to ensure that AI Agents are used in a way that benefits both businesses and workers'}, 'name': 'simple_search_agent'}}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 57, 'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 57}], 'prompt_token_count': 21115, 'prompt_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 21115}], 'total_token_count': 21172, 'traffic_type': <TrafficType.ON_DEMAND: 'ON_DEMAND'>}, 'invocation_id': 'e-54988485-7b90-4f04-82eb-3d004a64995e', 'author': 'research_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids': s

{'content': {'parts': [{'function_response': {'id': 'adk-1d5c6a89-edcc-4a2c-8ccb-62533dcd524c', 'name': 'simple_search_agent', 'response': {'result': "It's true that research and development efforts are essential to ensure AI agents benefit both businesses and workers. Here's a breakdown of why, and what areas these efforts should focus on:\n\n**1. Benefits of AI Agents in Business and Work:**\n\n*   **Increased Productivity & Efficiency:** AI agents can automate repetitive tasks (like data entry, scheduling, and basic research), freeing up human workers for more creative and strategic work. Some businesses have reported a potential 30% increase in operational efficiency.\n*   **Cost Savings:** Businesses using AI agents have reported cost reductions (as high as 30%) through handling more customer inquiries and boosting e-commerce efficiency. AI agents can also reduce the need for employee benefits and compensation for downtime.\n*   **Improved Decision-Making:** AI agents can quickly 

{'content': {'parts': [{'text': 'Okay, I have answered all the questions from the research plan.\n'}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 14, 'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 14}], 'prompt_token_count': 22081, 'prompt_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 22081}], 'total_token_count': 22095, 'traffic_type': <TrafficType.ON_DEMAND: 'ON_DEMAND'>}, 'invocation_id': 'e-54988485-7b90-4f04-82eb-3d004a64995e', 'author': 'research_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 'lm4vQhEK', 'timestamp': 1748879103.610679}


{'content': {'parts': [{'text': 'Okay, here\'s a research report in markdown format, summarizing the search results regarding the impact of AI Agents on the labor market.\n\n# Research Report: AI Agent\'s Impact on the Labor Market\n\n## 1. Foundational Understanding\n\n### 1.1. Definition and Types of AI Agents in the Labor Market\n\nAI agents are sophisticated software programs designed to autonomously perform tasks, make decisions, and interact with their environment with minimal human oversight. Key characteristics include autonomy, reasoning, learning, and interactivity.\n\n**Types of AI Agents:**\n\n*   **Chatbots:** Managing routine customer inquiries.\n*   **Robotic Process Automation (RPA):** Automating repetitive administrative tasks like data collection, document verification, and reporting.\n*   **AI Assistants:** Assisting in healthcare, finance, and HR tasks such as patient inquiry management, invoice matching, and payroll verification.\n*   **Autonomous Systems:** Managi

In [28]:
from IPython.display import display, Markdown

display(Markdown(response_text))

Okay, I can create a research plan to investigate the impact of AI Agents on the labor market. Here's a plan designed to guide a thorough exploration of the topic:

*   **Foundational Understanding**

    *   What is the definition of an AI Agent, and what are the different types of AI Agents relevant to the labor market (e.g., chatbots, robotic process automation, autonomous systems)?
    *   What are the key capabilities and limitations of current AI Agent technologies?
    *   What are the historical trends of automation and technological advancements and their effects on employment?
*   **In-Depth Analysis**

    *   How are AI Agents currently being implemented in various industries and sectors?
    *   What specific job roles and tasks are most susceptible to being augmented or replaced by AI Agents?
    *   What are the observed effects of AI Agent implementation on employment levels, both positive (job creation) and negative (job displacement)?
    *   How is the introduction of AI Agents affecting the skill requirements for various jobs?
    *   What are the effects of AI Agents on wages and income inequality?
    *   What are the psychological and sociological impacts on workers whose jobs are affected by AI Agents?
*   **Context and Connections**

    *   How does the impact of AI Agents on the labor market vary across different countries and economies? What are the influencing factors?
    *   What are the ethical considerations surrounding the use of AI Agents in the workplace, such as bias, privacy, and accountability?
    *   What are the existing legal and regulatory frameworks governing the use of AI Agents in employment?
    *   How do economic factors like productivity growth, investment, and consumer demand mediate the impact of AI Agents on employment?
    *   How does the development and deployment of AI Agents intersect with broader trends like globalization, remote work, and the gig economy?
*   **Practical Implications**

    *   What strategies can businesses and policymakers adopt to mitigate the negative impacts of AI Agents on workers and promote a smooth transition?
    *   What types of education and training programs can help workers develop the skills needed to complement or work alongside AI Agents?
    *   How can labor unions and worker organizations adapt to the changing landscape of work in the age of AI Agents?
    *   What innovative approaches to social safety nets and income support can help workers who are displaced by AI Agents? (e.g., universal basic income, retraining stipends)
*   **Future Outlook**

    *   What are the emerging trends and future directions in AI Agent technology, and how might these trends further impact the labor market?
    *   What are the potential long-term effects of widespread AI Agent adoption on the nature of work, the organization of companies, and the distribution of wealth?
    *   What are the key uncertainties and unknowns surrounding the future of AI Agents and their impact on the labor market?
    *   What research and development efforts are needed to ensure that AI Agents are used in a way that benefits both businesses and workers?
Okay, I will now conduct searches to answer the questions outlined in the research plan. Here are the search results, organized by question:

**Foundational Understanding**

*   **What is the definition of an AI Agent, and what are the different types of AI Agents relevant to the labor market (e.g., chatbots, robotic process automation, autonomous systems)?**
*   **What are the key capabilities and limitations of current AI Agent technologies?**
*   **What are the historical trends of automation and technological advancements and their effects on employment?**
**In-Depth Analysis**

*   **How are AI Agents currently being implemented in various industries and sectors?**
*   **What specific job roles and tasks are most susceptible to being augmented or replaced by AI Agents?**
*   **What are the observed effects of AI Agent implementation on employment levels, both positive (job creation) and negative (job displacement)?**
*   **How is the introduction of AI Agents affecting the skill requirements for various jobs?**
*   **What are the effects of AI Agents on wages and income inequality?**
*   **What are the psychological and sociological impacts on workers whose jobs are affected by AI Agents?**
**Context and Connections**

*   **How does the impact of AI Agents on the labor market vary across different countries and economies? What are the influencing factors?**
*   **What are the ethical considerations surrounding the use of AI Agents in the workplace, such as bias, privacy, and accountability?**
*   **What are the existing legal and regulatory frameworks governing the use of AI Agents in employment?**
*   **How do economic factors like productivity growth, investment, and consumer demand mediate the impact of AI Agents on employment?**
*   **How does the development and deployment of AI Agents intersect with broader trends like globalization, remote work, and the gig economy?**
**Practical Implications**

*   **What strategies can businesses and policymakers adopt to mitigate the negative impacts of AI Agents on workers and promote a smooth transition?**
*   **What types of education and training programs can help workers develop the skills needed to complement or work alongside AI Agents?**
*   **How can labor unions and worker organizations adapt to the changing landscape of work in the age of AI Agents?**
*   **What innovative approaches to social safety nets and income support can help workers who are displaced by AI Agents? (e.g., universal basic income, retraining stipends)**
**Future Outlook**

*   **What are the emerging trends and future directions in AI Agent technology, and how might these trends further impact the labor market?**
*   **What are the potential long-term effects of widespread AI Agent adoption on the nature of work, the organization of companies, and the distribution of wealth?**
*   **What are the key uncertainties and unknowns surrounding the future of AI Agents and their impact on the labor market?**
*   **What research and development efforts are needed to ensure that AI Agents are used in a way that benefits both businesses and workers?**
Okay, I have answered all the questions from the research plan.
Okay, here's a research report in markdown format, summarizing the search results regarding the impact of AI Agents on the labor market.

# Research Report: AI Agent's Impact on the Labor Market

## 1. Foundational Understanding

### 1.1. Definition and Types of AI Agents in the Labor Market

AI agents are sophisticated software programs designed to autonomously perform tasks, make decisions, and interact with their environment with minimal human oversight. Key characteristics include autonomy, reasoning, learning, and interactivity.

**Types of AI Agents:**

*   **Chatbots:** Managing routine customer inquiries.
*   **Robotic Process Automation (RPA):** Automating repetitive administrative tasks like data collection, document verification, and reporting.
*   **AI Assistants:** Assisting in healthcare, finance, and HR tasks such as patient inquiry management, invoice matching, and payroll verification.
*   **Autonomous Systems:** Managing inventory, predicting stock shortages, and reordering products automatically.

### 1.2. Key Capabilities and Limitations of Current AI Agent Technologies

**Capabilities:**

*   **Reasoning:** Drawing conclusions and solving problems based on available information.
*   **Acting:** Performing tasks based on decisions and external inputs.
*   **Autonomy:** Operating and making decisions independently.
*   **Learning:** Adapting and improving performance over time through machine learning.
*   **Task Automation:** Automating complex tasks at scale.
*   **Data Analysis:** Identifying patterns and delivering strategic insights.
*   **Tool Use:** Leveraging external tools, APIs, and systems.
*   **Interactivity:** Seamlessly interacting with digital interfaces.

**Limitations:**

*   **Errors and Malfunctions:** Prone to errors, especially in novel situations.
*   **Security Issues:** Vulnerable to cyberattacks.
*   **Dependence on Data:** Reliance on large, potentially biased datasets.
*   **Integration Complexities:** Challenges in integrating with existing systems.
*   **Lack of Creativity:** Limited in genuine creativity, intuition, and abstract thinking.
*   **Ethical Concerns:** Potential for bias, lack of accountability, and privacy violations.
*   **Decision-Making Limitations:** Struggles with nuanced conversations and emotional intelligence.
*   **Communication Challenges:** Limited communication between agents.

### 1.3. Historical Trends of Automation and Technological Advancements and Their Effects on Employment

Historical trends indicate a complex relationship between automation and employment, characterized by both job displacement and job creation.

*   **Job Displacement:** Automation of tasks, reduced demand for low-skill jobs, and skill-biased technological change.
*   **Job Creation:** Emergence of new industries and jobs, increased productivity, and economic growth.
*   **Transformation of Work:** Changes in job roles (e.g., bank tellers shifting to customer service), increased demand for high-order cognitive skills, and job quality improvements.

## 2. In-Depth Analysis

### 2.1. Current Implementation of AI Agents in Various Industries and Sectors

AI agents are being implemented across various sectors to automate tasks, enhance decision-making, and improve customer experiences.

*   **Business:** Automation of standardized processes, collaboration with humans, and data insights.
*   **Marketing:** Content creation and personalized customer interactions.
*   **Customer Service:** AI-driven chatbots and virtual assistants.
*   **Supply Chain Management:** Real-time inventory monitoring and automated reordering.
*   **Finance:** Fraud detection and financial analysis.
*   **Healthcare:** Patient inquiry management and workflow optimization.
*   **Human Resources:** Onboarding, internal mobility, and employee query resolution.

### 2.2. Specific Job Roles and Tasks Most Susceptible to Augmentation or Replacement by AI Agents

Job roles susceptible to AI augmentation or replacement include:

*   Data Entry Clerks
*   Telemarketers
*   Receptionists
*   Customer Service Representatives
*   Bookkeeping Clerks
*   Retail Cashiers
*   Manufacturing Workers
*   Administrative Assistants

Tasks include repetitive and administrative functions, such as data collection, document verification, compliance support, and internal reporting.

### 2.3. Observed Effects of AI Agent Implementation on Employment Levels

*   **Job Displacement:** Automation leads to job displacement, particularly in routine-based roles.
*   **Job Creation:** Emergence of new roles like AI trainers, data analysts, and AI maintenance professionals.
*   **Job Transformation:** Shifting focus to higher-value work, requiring adaptability and AI literacy.
*   **Overall Impact:** Potential net job gain, increased productivity, and economic growth, though concerns about inequality persist.

### 2.4. Impact of AI Agents on Skill Requirements for Various Jobs

AI is shifting skill requirements, emphasizing technical skills and human qualities.

*   **Automation of Routine Tasks:** Workers focus on complex, creative, and strategic work.
*   **Creation of New Job Roles:** Demand for AI Ethics Specialists, Data Scientists, and Machine Learning Engineers.
*   **Shift in Skill Requirements:** Need for AI-specific proficiencies like programming and data analysis.
*   **Increased Emphasis on Human Skills:** Importance of empathy, critical thinking, and emotional intelligence.

### 2.5. Effects of AI Agents on Wages and Income Inequality

AI's impact on wages and income inequality is complex:

*   **Job Losses:** Potential displacement of millions of jobs.
*   **New Job Creation:** Creation of new job opportunities in AI-related fields.
*   **Wage Stagnation/Decline:** Possible decline in wages for some workers due to automation.
*   **Exacerbating Inequality:** AI could worsen income inequality by favoring high-income workers.
*   **Need for Upskilling/Reskilling:** Mitigating negative impacts through training programs and adaptability.

### 2.6. Psychological and Sociological Impacts on Workers

*   **Anxiety and Stress:** Job security concerns, increased stress, and feelings of inadequacy.
*   **Impact on Mental Health:** Negative effects on mental health, including depression and social isolation.
*   **Self-Esteem and Purpose:** Reduced self-esteem and loss of meaning.
*   **Changes in Work Environment:** Weaker team bonds and increased competition.
*   **Potential Benefits:** Reduced workload and improved job engagement.
*   **Job Displacement and Insecurity:** Leading to anxiety and reduced job satisfaction.
*   **Economic Inequality:** Concentration of wealth and a shrinking middle class.
*   **Deskilling:** Potential deskilling of some roles.
*   **Reduced Social Interaction:** Negative impact on employee morale and well-being.

## 3. Context and Connections

### 3.1. Variation of AI Agent Impact Across Countries and Economies

The impact of AI varies based on technological infrastructure, levels of development, and industry types:

*   **High-Income Countries:** Higher employment exposure due to advanced infrastructure.
*   **Low-Income Countries:** Lower risk of job displacement in the short term but risk of falling behind.
*   **Manufacturing-Based Economies:** Significant job displacement in routine tasks.
*   **Service-Based Economies:** Shift in job roles and new opportunities in AI development.

### 3.2. Ethical Considerations

*   **Job Displacement:** Ethical implementation, training programs, and regulations are crucial.
*   **Bias and Discrimination:** Use diverse data sets and monitor AI models for bias.
*   **Accountability and Transparency:** Establish AI governance structures and assign responsibility for AI actions.
*   **Privacy and Surveillance:** Balance privacy against productivity demands.
*   **Human Dignity and Well-being:** Ensure AI enhances human capabilities rather than replaces them.

### 3.3. Legal and Regulatory Frameworks

*   Regulations are lagging behind AI development.
*   Key concerns include accountability, data privacy, bias, and transparency.
*   Emerging frameworks include the EU AI Act and AI Agent Compliance Frameworks.

### 3.4. Mediation by Economic Factors

Economic factors mediate AI's impact:

*   **Productivity Growth:** Boosts demand for labor and economic growth.
*   **Investment:** Drives AI development and job creation.
*   **Consumer Demand:** Influences the adoption and impact of AI.

### 3.5. Intersection with Globalization, Remote Work, and the Gig Economy

*   **Globalization:** AI enhances communication and customer experience across borders.
*   **Remote Work:** AI automates project coordination and provides support for remote teams.
*   **Gig Economy:** AI enables talent platforms to manage a growing user base and matches freelancers with projects.

## 4. Practical Implications

### 4.1. Strategies for Businesses and Policymakers

*   **Workforce Transition & Support:** Retraining programs, job redesign, and support for displaced employees.
*   **Ethical & Transparent AI Implementation:** Transparency, fairness, and open communication.
*   **Culture of Adaptability & Learning:** Continuous learning and a growth mindset.
*   **Addressing Employee Concerns & Well-being:** Managing workload and promoting mental fitness.
*   **Government & Policy Measures:** Encouraging complementary innovation and investing in education.

### 4.2. Education and Training Programs

*   Technical literacy, strategic oversight, AI-specific skills, and "human" skills.
*   Upskilling and reskilling initiatives, personalized learning, and hands-on training.

### 4.3. Adaptation for Labor Unions and Worker Organizations

*   Embrace AI tools, advocate for worker rights, focus on skills development, and foster collaboration.

### 4.4. Innovative Approaches to Social Safety Nets

*   Universal Basic Income (UBI), AI Displacement Insurance (AIDI), portable benefits, and wage insurance.

## 5. Future Outlook

### 5.1. Emerging Trends and Future Directions

*   Increased automation, AI-human collaboration, personalized experiences, and new roles.

### 5.2. Potential Long-Term Effects

*   **Nature of Work:** Increased productivity, evolving skills, and improved workplace experience.
*   **Organization of Companies:** Flatter hierarchies and increased agility.
*   **Distribution of Wealth:** Increased wealth inequality.

### 5.3. Key Uncertainties and Unknowns

*   Job displacement and creation, skill shifts, ethical considerations, and economic dynamics.

### 5.4. Needed Research and Development Efforts

*   Ethical considerations (bias mitigation, transparency, accountability), addressing job displacement (reskilling programs, job redesign), and data quality management.

This report provides a comprehensive overview of the multifaceted impact of AI Agents on the labor market, highlighting both the opportunities and challenges that lie ahead. Further research and proactive strategies are essential to ensure a smooth and equitable transition into the age of AI.


## Bonus: Finish the remaining researches

Note that the above result has many open questions not yet researched. Try to build an agent tree that completes the research. Some possible directions:

- Use LoopAgent, refer to the QuickStart notebook of how to exit_loop.
- Optimize the prompt of multiple_search agent so it conducts more thorough searches.

## Bonus: Format with grounding data

The simple_search_agent returns not only the search results summary, but also the grounding web pages and references / citations.